# Classification - Feature engineering
This notebook contains an implementation of the maintenance train dataset ML problem as a multi-class classification problem creating and transforming features from the dataset to achieve best results

This will be achieved by creating different steps in the ML-pipeline that take parameters as input and can produce different attributes from the dataset which will be passed into the ML model:

In [85]:
import pandas as pd
import numpy as np
import ast

from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer, TfidfVectorizer
from sklearn.model_selection import cross_val_score, LeaveOneOut
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, f1_score
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectFromModel
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
import warnings

# Supress warnings that appear only after MultiLabelBinarizer encoding event codes that are only present in the test data
warnings.filterwarnings("ignore", message="unknown class(es) .*")
pd.options.display.max_columns = None
pd.options.display.max_rows = None

Loading the dataset:

In [4]:
df = pd.read_csv('sncb_data_challenge.csv', delimiter=';')
df.events_sequence = df.events_sequence.apply(ast.literal_eval) #transforming column saved as string to list of elements
df.seconds_to_incident_sequence = df.seconds_to_incident_sequence.apply(ast.literal_eval) #transforming column saved as string to list of elements
df.train_kph_sequence = df.train_kph_sequence.apply(ast.literal_eval) #transforming column saved as string to list of elements
df.dj_ac_state_sequence = df.dj_ac_state_sequence.apply(ast.literal_eval)
df.dj_dc_state_sequence = df.dj_dc_state_sequence.apply(ast.literal_eval)
df.vehicles_sequence = df.vehicles_sequence.apply(ast.literal_eval)
df.sample(2)

,incident_id,vehicles_sequence,events_sequence,seconds_to_incident_sequence,approx_lat,approx_lon,train_kph_sequence,dj_ac_state_sequence,dj_dc_state_sequence,incident_type
675,4464675,"[566, 566, 566, 566, 566, 566, 566, 566, 566, ...","[4066, 3636, 3658, 2684, 2682, 4124, 2956, 295...","[-14370, -14347, -14347, -14325, -14323, -1432...",51.156479,4.140405,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 53.1, 112.7, 10...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",2
21,4434099,"[563, 563, 563, 563, 563, 563, 563, 563, 563, ...","[1566, 1570, 4394, 4396, 4016, 4026, 4026, 402...","[-13011, -13011, -13010, -12916, -7762, -7762,...",51.101056,4.635386,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[False, False, False, False, False, False, Fal...","[True, True, True, True, True, True, True, Tru...",4


## Cleaning frequent events
Next we remove the most frequent events from `events_sequence`. Those concern events that occur in at least $per$ % of all incidents:

Create class in order to optimize the percentage threshold of frequent events to be filtered. This class will be a separate step in the ML pipeline

This class takes as a parameter a percentage $per$. This percentage is the threshold that concerns the most frequent events. Frequent events are defined as events that happen at least once in most incidents. For example, a threshold of 85%, will filter out all the event codes that happen at least once in more than 85% of the incidents.

By doing that, we will optimize this $per$ parameter with fine-tuning to find the percentage that produces the best results for our metric. This filtering only concerns the tf-idf vectorizer: 

In [7]:
class OptimizeCleanFrequentEvents(BaseEstimator, TransformerMixin):
    def __init__(self, per=100):
        self.per = per
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        events_types_dict = {}
        for events_sequence in X_new['events_sequence']:
            row_list = events_sequence
            unique_events = set(row_list)
            for event in unique_events:
                if not events_types_dict.get(event):
                    events_types_dict[event] = 0
                events_types_dict[event] += 1
        sorted_dict = dict(sorted(events_types_dict.items(), key=lambda item: item[1], reverse=True))
        sorted_events_perc_df = pd.DataFrame(list(sorted_dict.items()), columns=['event_type', 'frequency'])
        sorted_events_perc_df['percentage'] = sorted_events_perc_df['frequency'] / X.shape[0] * 100
        sorted_events_perc_df['event_type'] = sorted_events_perc_df['event_type'].astype(str)
        events_low_frequency = list(map(int, list(sorted_events_perc_df[sorted_events_perc_df.percentage<=self.per].event_type)))
        X_new['clean_events_sequence'] = X.events_sequence.apply(lambda x: [i for i in x if i in events_low_frequency]).astype(str)\
                .replace(r'[\[\],]', '', regex=True)
        return X_new

## Number of vehicles
As explored in the exploratory analysis some incidents are connected to train sequences that consist of multiple vehicles

That's why one attribute we create is the number of vehicles for that particular incident

In [9]:
# Transform to set to remove duplicates and then calculate number of unique items of set
df['number_vehicles'] = df['vehicles_sequence'].apply(set).apply(len)

Let's check the `events_sequence`, `vehicles_sequence`, `seconds_to_incident_sequence` of an incident with multiple vehicles. Unfortunately, the seconds are not ordered which means that the various lists do not have the events in a ordered chronological way but they are only ordered for the same vehicles. Meaning seconds are ordered for the first vehicles and then right after follow the ordered seconds for the second vehicle and so on. We can see that in an example with the second incident :

In [11]:
pd.DataFrame({
        'vehicles_sequence': df.loc[1, 'vehicles_sequence'],
        'seconds_to_incident_sequence': df.loc[1, 'seconds_to_incident_sequence'],
        'events_sequence': df.loc[1, 'events_sequence'],
    })

,vehicles_sequence,seconds_to_incident_sequence,events_sequence
0,526,-8573,2744
1,526,-8573,4148
2,526,-8032,4394
3,526,-8032,1566
4,526,-8032,1570
5,526,-7859,4396
6,526,-6146,3634
7,526,-5862,4124
8,526,-5651,2956
9,526,-5634,2956


We will order all seconds to the actual chronological order and together with that we will order correspondingly all the other list attributes since those are connected to the particular timestamp (seconds). For example, the lists `seconds_to_incident_sequence` [-5, -1, 1] and `events_sequence` [4394, 1566, 1570] mean that:
- The event 4394 happened -5 sec before the incident
- The event 1566 happened -1 sec before the incident
- The event 1570 happened 1 sec after the incident

The idea for doing this is that we will try to create new attributes that will contains specific information right before the incident happened (time: 0 sec). For multi-vehicle incidents that would be possible since those were not sorted chronologically:

In [13]:
# Function to sort seconds_to_incident_sequence and reorder other columns accordingly
def sort_and_reorder(row):
    # Sort column and get the sorted indices while keeping the current order for equal elements
    sorted_indices = np.lexsort((np.arange(len(row['seconds_to_incident_sequence'])), row['seconds_to_incident_sequence']))
    
    # Apply the sorted indices to all desired columns
    sorted_seconds = [row['seconds_to_incident_sequence'][i] for i in sorted_indices]
    reordered_vehicles = [row['vehicles_sequence'][i] for i in sorted_indices]
    reordered_events = [row['events_sequence'][i] for i in sorted_indices]
    reordered_speed = [row['train_kph_sequence'][i] for i in sorted_indices]
    reordered_dj_ac = [row['dj_ac_state_sequence'][i] for i in sorted_indices]
    reordered_dj_dc = [row['dj_dc_state_sequence'][i] for i in sorted_indices]
       
    
    return pd.Series([sorted_seconds, reordered_vehicles, reordered_events, reordered_speed,
                     reordered_dj_ac, reordered_dj_dc],
                     index=['seconds_to_incident_sequence', 'vehicles_sequence', 'events_sequence',
                               'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence'])
df[['seconds_to_incident_sequence', 'vehicles_sequence', 'events_sequence',
   'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence']] =df.apply(sort_and_reorder, axis=1)

### Final events before incident

Now we will create some extra attributes that will describe the last events before the actual occurence of the incident. To do that we will check the `seconds_to_incident_sequence`, and find the list index of the last timestamp before the incident which is the the index of largest negative second in `seconds_to_incident_sequence`. Saving those indexes we will be able to search and find event right before the incident happening.

In [15]:
# Function to find the index of the maximum negative value
def max_negative_index(lst):
    # Filter out negative values and their indices
    neg_values = [(i, x) for i, x in enumerate(lst) if x < 0]
    max_neg_index, max_neg_value = max(neg_values, key=lambda x: x[1])
    return max_neg_index

Find max negative index. Practically since the events are sequential, we only need to find the max negative index. Knowing that, we can find the other events and seconds since the same index of a list in `seconds_to_incident_sequence` and `events_sequence` correspond to the same event and `seconds_to_incident_sequence` are sorted chronologically:

In [17]:
df['max_negative_index'] = df.seconds_to_incident_sequence.apply(max_negative_index)

Now we use one hot encoding to those events to create extra columns that show the presence or absence of those events in the last n events before incident:

We create another class to be passed in the parameter tuning in order to optimize $last\_events$, the number of the last events before the incident. For example, $last\_events =3$ would mean that the three last events before the incident are encoded using MultiLabelBinarizer (since each sample incident can have multiple previous events) and added to the existing attributes

In [20]:
class OptimizeLastEventsNumber(BaseEstimator, TransformerMixin):
    def __init__(self, last_events=1):
        self.last_events = last_events
        self.mlb = MultiLabelBinarizer()
        
    def fit(self, X, y=None):
        X_new = X.copy()
        X_new['event_last_n'] = X_new.apply(lambda row: row['events_sequence'][row['max_negative_index']-self.last_events+1: row['max_negative_index']+1],
                                            axis=1).apply(lambda x: [str(i) for i in x]) # Extract the minus ith element of the column list
        # Fit the MultiLabelBinarizer only on the specified column
        self.mlb.fit(X_new['event_last_n'])
        return self
    
    def transform(self, X):
        X_new = X.copy()
        X_new['event_last_n'] = X_new.apply(lambda row: row['events_sequence'][row['max_negative_index']-self.last_events+1: row['max_negative_index']+1],
                                            axis=1).apply(lambda x: [str(i) for i in x]) # Extract the minus ith element of the column list
        one_hot_encoded = self.mlb.transform(X_new['event_last_n'])
        one_hot_df = pd.DataFrame(one_hot_encoded, columns=self.mlb.classes_, index=X.index)
        # Concatenate the original DataFrame (without the transformed column) and the MultiLabelBinarizer DataFrame
        X_transformed = pd.concat([X, one_hot_df], axis=1)
        return X_transformed

## Train speed before incident
Another attribute will be created using the train speed sequence. Again, we use the `max_negative_index` attribute to detect the train speeds right before the occurrence of the incident
We will again use a parameter $last\_events$ to identify the last_events speeds to be used. Since the speeds correspond to different timestamps we will take the mean speed in the defined last_events:

Again we create another class to optimize this parameter

In [23]:
class OptimizeLastTrainSpeeds(BaseEstimator, TransformerMixin):
    def __init__(self, last_events=1):
        self.last_events = last_events
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        if self.last_events > 0:
            X_new['average_speed_last_events'] = X_new.apply(lambda row: sum(row['train_kph_sequence'][row['max_negative_index']-self.last_events+1:row['max_negative_index']+1]) / self.last_events,  axis=1)
        return X_new

## Powerlines attributes
Next we can also create some attributes regarding the columns `dj_ac_state_sequence` and `dj_dc_state_sequence` which describe the two pantographs of the train. Usually, just one or 0 is up.

Another class to optimize the number of states before the incident. This step in the pipeline will create double the number of boolean attributes as $last\_events$ defined since there will be one boolean variable for each state for each last step :

In [26]:
class OptimizeLastStates(BaseEstimator, TransformerMixin):
    def __init__(self, last_events=1):
        self.last_events = last_events
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        for i in range(self.last_events):
            column_name_ac = 'dj_ac_state_last_' + str(i+1)
            column_name_dc = 'dj_dc_state_last_' + str(i+1)
            X_new[column_name_ac] = X_new.apply(lambda row: row['dj_ac_state_sequence'][row['max_negative_index'] - i], axis=1)
            X_new[column_name_dc] = X_new.apply(lambda row: row['dj_dc_state_sequence'][row['max_negative_index'] - i], axis=1)
        return X_new

## Emergency breaking feature
Now we can calculate a new variable that contains if there was an emergency breaking in the last $n$ events before the incident happened:

Another class to optimize is the check whether there was an emergency break in the $last\_events$ number of events before the incident. This step in the pipeline will create a new boolean attribute that is True if there was an emergency break in the passed $last\_events$

In [29]:
class OptimizeLastCheckEmergencyBreak(BaseEstimator, TransformerMixin):
    def __init__(self, last_events=10):
        self.last_events = last_events
    
    def fit(self, X, y=None):
        return self
        
    def check_value_before_index(self, lst_event, value, index):
        return value in lst_event[index-self.last_events+1:index+1]
        
    def transform(self, X):
        X_new = X.copy()
        if self.last_events > 0:
            X_new['emergency_breaking_last_events'] = X_new.apply(lambda row: self.check_value_before_index(row['events_sequence'], 4080, row['max_negative_index']), axis=1)
        return X_new

## Event sequence
We will calculate the tf-idf in the `events_sequence` column, which calculates how relevant an event is in a sequence. The meaning increases proportionally to the number of times an event appears in a sequence but is compensated by the event frequency in the sequence. Firstly we remove the commas and the brackers from the events_sequence column

Since we use multiple different attributes and not all of them should be passed in  the TfidfVectorizer, we will define a preprocessor:

In [31]:
# Define the column transformer to apply TfidfVectorizer only to 'clean_events_sequence'
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', TfidfVectorizer(ngram_range=(1,1)), 'clean_events_sequence')
    ],
    remainder='passthrough'  # This passes the other columns without modification
)

## Remove attributes
As another class, we have one that just removes the unnecessary attributes after the desired attributes have been created from them

In [33]:
class RemoveAttributes(BaseEstimator, TransformerMixin):
    def __init__(self):
        self
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X_new = X.copy()
        return X_new.drop(['incident_id', 'vehicles_sequence', 'events_sequence',
                           'seconds_to_incident_sequence', 'approx_lat', 'approx_lon',
                           'train_kph_sequence', 'dj_ac_state_sequence', 'dj_dc_state_sequence',
                           'incident_type', 'max_negative_index'], axis=1) 

Also a class that helps debug is the print attributes class:

In [35]:
class PrintAttributes(BaseEstimator, TransformerMixin):
    def __init__(self):
        self
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        print(X.columns)
        return X

## Metric
As a metric f1 score is used. We define a custom f1 function and scorer that allows zero division and calculated the weighted f1 score which accounts for label imbalance

In detail F1 is calculated as:
$$ F1 Score= 2×\frac{Precision×Recall}{Precision+Recall}
​
$$
There are some minority classes with no correct predictions ($recall=0$) resulting in a null value for the whole f1 score when using a non-custom f1 scorer.

In [37]:
# Create a custom f1 function with zero_division parameter
def custom_f1_score(y_true, y_pred):
    return f1_score(y_true, y_pred, average='weighted', zero_division=0)

# Wrap the custom scoring function using make_scorer
f1_scorer = make_scorer(custom_f1_score)

## Random-searchCV for parameter tuning
Next step is the parameter tuning of the pipeline

First we define different sampling strategies to tackle the issue of the imbalanced classes.

In [39]:
target = df['incident_type'].copy() # target column separated
class_counts = target.value_counts()
max_class_count = max(class_counts.values)
# Each non-majority class will have equal samples to n% of the majority class plus their previous samples (n can be checked from the sampling strategy name)
sampling_strategy_grid_10 = {class_counts.index[i]: int(max_class_count * 0.1) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}
sampling_strategy_grid_15 = {class_counts.index[i]: int(max_class_count * 0.15) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}
sampling_strategy_grid_20 = {class_counts.index[i]: int(max_class_count * 0.2) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}
sampling_strategy_grid_25 = {class_counts.index[i]: int(max_class_count * 0.25) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}
sampling_strategy_grid_30 = {class_counts.index[i]: int(max_class_count * 0.30) + class_counts.values[i]
                     for i in range(len(class_counts.index)) if class_counts.values[i] < max_class_count}

The pipeline is defined here with the set parameters that we do not want to alter:

In [41]:
pipeline_clf = Pipeline([
                    ('last_event_num_columns', OptimizeLastEventsNumber()),
                    ('last_train_speeds_columns', OptimizeLastTrainSpeeds()),
                    ('last_states_columns', OptimizeLastStates()),
                    ('last_check_emergency_break', OptimizeLastCheckEmergencyBreak()),
                    ('clean_frequent_events', OptimizeCleanFrequentEvents(per=85)),
                    ('remove_attributes', RemoveAttributes()),
                    #('print_attr', PrintAttributes()),
                    ('tf_idf', preprocessor),
                    ('smote', SMOTE(random_state=1, k_neighbors=2)),
                    #('oversampler', RandomOverSampler(sampling_strategy='auto', random_state=1)),
                    ('feature_selection', SelectFromModel(ExtraTreesClassifier(random_state=1))),
                    ('extra_trees', ExtraTreesClassifier(random_state=1)),
                    ])

Now we can define all the parameters that we want to fine tune:

In [43]:
param_grid = {
    'last_event_num_columns__last_events': range(0,10),
    'last_train_speeds_columns__last_events': range(0,5),
    'last_states_columns__last_events': range(0,5),
    'last_check_emergency_break__last_events': [0,10,11,12,13,14,15],
    #'clean_frequent_events__per': range(50,100),
    'smote__sampling_strategy': [sampling_strategy_grid_10, sampling_strategy_grid_15, sampling_strategy_grid_20,
                                sampling_strategy_grid_25, sampling_strategy_grid_30],
    #'feature_selection__estimator__n_estimators': [100, 150, 200 , 250, 300],  # Number of trees for feature selection
    #'feature_selection__threshold': ["mean", "median"],  # Feature importance threshold
    'extra_trees__n_estimators': [150, 200, 250, 300],        # Number of trees in ExtraTrees
    'extra_trees__max_depth': [None, 10, 20, 30],        # Depth of each tree
    'extra_trees__criterion': ['gini', 'entropy', 'log_loss'],
    #'extra_trees__max_depth': [None, 10, 20, 30],           # Max depth of each tree
    #'extra_trees__min_samples_split': [2, 5, 10],           # Minimum samples required to split a node
    #'extra_trees__min_samples_leaf': [1, 2, 4],             # Minimum samples required at a leaf node
    'extra_trees__max_features': ['sqrt', 'log2'], # The number of features to consider for best split
    #'extra_trees__bootstrap': [True, False],                # Whether bootstrap samples are used
}

Last step is to run the randomizedsearchCV. Instead of gridsearch, this method is used since the number of hyperparameters is large and it doesn't attempt all combinations, reducing computational load and increasing efficiency. Of course the final parameters are not guaranteed to be the best but when a high number of iterations is set, the results can be very good:

In [45]:
# Cross-validation strategy set here to replicate results
cv = StratifiedKFold(n_splits=4, shuffle=True, random_state=4)

random_search = RandomizedSearchCV(pipeline_clf, param_grid, n_iter=20, cv=cv, n_jobs=-1, verbose=3, scoring=f1_scorer,
                                   random_state=4, error_score='raise')

random_search.fit(df, target)

# Print the best parameters and the best score
print("Best Parameters:", random_search.best_params_)
print("F1:", random_search.best_score_)

Fitting 4 folds for each of 20 candidates, totalling 80 fits
Best Parameters: {'smote__sampling_strategy': {99: 254, 14: 228, 2: 198, 9: 196, 4: 157, 11: 105, 17: 89, 6: 85, 3: 84, 16: 83, 7: 83}, 'last_train_speeds_columns__last_events': 0, 'last_states_columns__last_events': 4, 'last_event_num_columns__last_events': 6, 'last_check_emergency_break__last_events': 0, 'extra_trees__n_estimators': 200, 'extra_trees__max_features': 'sqrt', 'extra_trees__max_depth': None, 'extra_trees__criterion': 'gini'}
F1: 0.6548976917398965


Last, also the accuracy is showcased:

In [46]:
print("Accuracy:", str(np.mean(cross_val_score(random_search.best_estimator_, df, target, cv=cv, scoring='accuracy'))))

C:\Users\stef4\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['1090', '1142', '1224', '1252', '1418', '148', '1578', '1666', '1722', '1726', '1728', '1730', '1780', '1876', '200', '2008', '210', '2108', '2148', '2198', '2288', '2354', '2368', '2530', '2780', '2782', '2800', '2802', '2884', '2988', '3410', '3512', '3516', '3532', '3662', '3674', '3680', '3688', '408', '448', '570', '684', '734', '794', '800'] will be ignored
  warnings.warn(
C:\Users\stef4\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWarning: unknown class(es) ['1072', '1094', '1106', '1124', '1484', '1622', '1640', '1642', '1646', '1648', '1752', '1800', '1820', '1840', '1860', '1962', '2048', '2088', '2508', '2518', '2552', '2960', '2986', '2992', '3052', '3080', '3232', '3492', '374', '3994', '406', '538', '736', '86', '930'] will be ignored
  warnings.warn(
C:\Users\stef4\anaconda3\Lib\site-packages\sklearn\preprocessing\_label.py:900: UserWar

Accuracy: 0.6716426689252777
